Este projeto destina-se a criar um modelo de machine learning capaz de classificar as tags das perguntas do site stackoverflow.

# Imports

In [1]:
!pip install scikit-learn==0.24.1

  Using cached scikit-learn-0.24.1.tar.gz (7.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-learn


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [55 lines of output]
      Partial import of sklearn during the build process.
      <string>:116: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
      INFO: No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
      Traceback (most recent call last):
        File "C:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 363, in <module>
          

In [2]:
!pip install scikit-multilearn==0.2.0 

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss

from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.adapt import MLkNN

# Coleta de dados

In [4]:
perguntas = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/alura_classificacao_multilabel/master/dataset/stackoverflow_perguntas.csv')

# Análise Exploratória

In [5]:
# sample dos dados para investigação inicial
perguntas.sample(10)

,Perguntas,Tags
1801,Tenho a seguinte estrutura: CODE CODE ...,html
2035,Eu tenho um projeto HTML5 Cordova e eu preciso...,node.js
2831,Estou fazendo uma requisição ajax assim: CODE...,jquery
5058,Estou tentando validar meus campos com require...,html
2385,Estou fazendo uma aplicação em que o CODE pos...,angular
4306,Como bloquear o scroll whell do mouse em uma C...,jquery
2716,"Galera é o seguinte, comecei um novo projeto e...",angular
4696,Tenho um carousel que é criado dinamicamente c...,jquery html
1119,"Estou desenvolvendo um site (MVC), onde nele h...",html
3425,Como melhorar este script? A função CODE dev...,jquery


Nas tags, há mais que uma característica para algumas perguntas, trata-se então de uma coluna multilabel.

In [6]:
print(len(perguntas))

5408


Há 5408 perguntas no dataframe.

## Quantas combinações de tags há no dataframe?

In [7]:
perguntas['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [8]:
perguntas['Tags'].nunique()

37

Há 37 combinações de tags, onde muitas delas, há as mesmas tecnologias a serem consideradas pois espaços tornam strings diferentes umas das outras ou tecnologias invertidas, ainda que possuam o mesmo conteúdo.

# Transformando strings em labels numéricas

In [9]:
# cria lista vazia que armazenará as labels
lista_de_tags = list()
# seleciona as tags que estão na array 
for tags in perguntas['Tags'].unique():
    # divide as strings em strings únicas 
    for tag in tags.split():
        # se alguma dessas strings não estiver na lista vazia, coloque-a dentro dela
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)

# mostra a lista
print(lista_de_tags)

['node.js', 'jquery', 'html', 'angular']


In [10]:
# # cria lista de dummy node_js
# node_js = list()
# # para cada uma das linhas, se houver a string 'node.js', 1, se não, 0
# for linha_tag in perguntas['Tags']:
#     if 'node.js' in linha_tag:
#         node_js.append(1)
#     else:
#         node_js.append(0)
# # criando coluna node.js no dataframe
# perguntas['node_js'] = node_js

Seria necessário fazer isso para cada uma das tags, portanto faz-se necessário a criação de uma função.

In [11]:
# função que cria colunas dummies para cada tag
def nova_coluna(lista_tags, dataframe, nome_tags):
    # lendo a tag na lista de tags obtidas na iteração anterior
    for tag in lista_tags:
        # lista vazia que armazenará os dados da coluna
        coluna = list()
        # lendo as linhas no dataframe que possuem as tags
        for linha_tag in dataframe[nome_tags]:
            # caso a tag esteja nessa linha, 1, se não, 0
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        # após criada a lista com as dummies, insere-a como coluna do dataframe
        dataframe[tag] = coluna

In [12]:
# criando colunas dummies com as tags
nova_coluna(lista_tags = lista_de_tags, dataframe = perguntas, nome_tags = 'Tags')

In [13]:
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [14]:
# criando lista de tuplas com as 4 colunas target
lista_zip_tags = list(zip(perguntas[lista_de_tags[0]],
                     perguntas[lista_de_tags[1]],
                     perguntas[lista_de_tags[2]],
                     perguntas[lista_de_tags[3]]))

# criando coluna no dataframe
perguntas['todas_tags'] = lista_zip_tags

In [15]:
list1 = [1, 2]
list2 = [3, 4]
list3 = list(zip(list1, list2))
print(list3)

[(1, 3), (2, 4)]


In [16]:
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


In [17]:
# dividindo o dataframe em dataframes de treino e teste
perguntas_treino, perguntas_test, tags_treino, tags_teste = train_test_split(perguntas['Perguntas'], perguntas['todas_tags'],
                                                                            test_size = 0.3, random_state = 123)               

# Vetorização

In [18]:
# instanciando o vetorizador
# max_features: parâmetro que limita o número de features do vetor
# max_df: desconsidera palavras com muita frequência
vetorizar = TfidfVectorizer(max_features = 5000, max_df = 0.85)

In [19]:
# treinando o vetorizador
vetorizar.fit(perguntas['Perguntas'])

# transformando vetores
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_test_tfidf = vetorizar.transform(perguntas_test)

In [20]:
print(perguntas_treino_tfidf.shape)
print(perguntas_test_tfidf.shape)

(3785, 5000)
(1623, 5000)


# One vs Rest Classifier

In [21]:
# instanciando o classificador com o estimador como sendo a regressão logística
regressao_logistica = LogisticRegression(solver = 'lbfgs')
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest

OneVsRestClassifier(estimator=LogisticRegression())

Para utilizar o classificador, será necessário transformar as séries criadas em arrays.

In [22]:
# transformando as séries do Pandas em listas e em seguida em numpy arrays
tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))


In [23]:
# treinando o modelo
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)

OneVsRestClassifier(estimator=LogisticRegression())

# Exact Match

In [24]:
# score do classificador
resultado_onevsrest = classificador_onevsrest.score(perguntas_test_tfidf, tags_teste_array)
print('Exact match: {0: .2f}%'.format(resultado_onevsrest*100))

Exact match:  39.68%


In [25]:
perguntas['todas_tags'].unique()

array([(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1),
       (0, 0, 0, 1), (0, 1, 1, 0), (1, 0, 0, 1), (0, 1, 1, 1),
       (0, 1, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0), (1, 0, 1, 1),
       (1, 1, 1, 0)], dtype=object)

In [26]:
perguntas['todas_tags'].nunique()

13

Apesar do modelo ter aparente baixa acurácia, a probabilidade de acerto aleatório seria de 1 em 13 (aproximadamente 7%), dessa forma, o modelo já desempenha mais que o esperado.

# Hamming Loss

In [27]:
previsao_onevsrest = classificador_onevsrest.predict(perguntas_test_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_teste_array, previsao_onevsrest)


In [28]:
print('Hamming loss: {0: .2f}'.format(hamming_loss_onevsrest))

Hamming loss:  0.19


O hamming loss é uma métrica que varia de 0 a 1, sendo cada vez melhor quanto mais próximo de zero.

# Classifier Chain

In [29]:
# instanciando o classificador em cadeia com a regressão logística como parâmetro
classificador_cadeia = ClassifierChain(regressao_logistica)

In [30]:
# obtendo exact match para a classificação em cadeia
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia =  classificador_cadeia.score(perguntas_test_tfidf, tags_teste_array)
print('Exact Match: {0: .2f}%'.format(resultado_cadeia*100))

# obtendo o hamming loss para a classificação em cadeia
previsao_cadeia = classificador_cadeia.predict(perguntas_test_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, previsao_onevsrest)
print('Hamming loss: {0: .2f}'.format(hamming_loss_cadeia))

Exact Match:  49.35%
Hamming loss:  0.19


A acurácia do classificador aumentou com relação ao classificador onevsrest, com o hamming loss igual ao anterior, então este modelo mostra-se mais promissor.

# Relevância Binária

In [31]:
# instanciando o classificador de relevância binária com a regressão logística como parâmetro
classificador_br = BinaryRelevance(regressao_logistica)

In [32]:
# obtendo exact match para a classificação com relevância binária
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br =  classificador_br.score(perguntas_test_tfidf, tags_teste_array)
print('Exact Match: {0: .2f}%'.format(resultado_br*100))

# obtendo o hamming loss para a classificação com relevância binária
previsao_br = classificador_br.predict(perguntas_test_tfidf)
hamming_loss_br = hamming_loss(tags_teste_array, previsao_onevsrest)
print('Hamming loss: {0: .2f}'.format(hamming_loss_br))

Exact Match:  39.68%
Hamming loss:  0.19


Este modelo mostrou-se igual ao classificador onevsrest.

*Conclusão:* O classificador em cadeia gerou resultados melhores na classificação para este projeto e com isso, somos capazes de classificar as tags das perguntas feitas no stackoverflow.